# Exercises - The Carry Trade

#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$

Use the data set `famabliss_strips_2025-11-28.xlsx`.

It gives prices on **zero coupon bonds** with maturities of 1 through 5 years.
* These are prices per \$1 face value on bonds that only pay principal.
* Such bonds can be created from treasuries by *stripping* out their coupons.
* In essence, you can consider these prices as the discount factors $Z$, for maturity intervals 1 through 5 years.

In this problem, we focus on six dates: the month of **November** in 2020 through 2025.

In [2]:
import numpy as np
import pandas as pd

DATA_PATH = "famabliss_strips_2025-11-28.xlsx"

df = pd.read_excel(DATA_PATH, sheet_name="prices")
df["date"] = pd.to_datetime(df["date"])

#  Nov 2020–2025 for total of 6 months
nov = (
    df[df["date"].dt.month == 11]
    .loc[df["date"].dt.year.between(2020, 2025)]
    .set_index("date")
    .sort_index()
)

display(nov)


,1,2,3,4,5
date,,,,,
2020-11-30,0.998839,0.997047,0.994464,0.988809,0.981515
2021-11-30,0.997523,0.988614,0.974796,0.958322,0.943772
2022-11-30,0.954375,0.918220,0.887608,0.857356,0.830593
2023-11-30,0.951080,0.911951,0.876957,0.842191,0.809378
2024-11-29,0.959187,0.920923,0.885440,0.850418,0.817795
2025-11-28,0.964936,0.932866,0.900919,0.868240,0.836057


# The Carry Trade

## 1.1

Suppose it is `November 2020`, and you determine to implement a carry trade with the following specification...

* Long `$100` million (market value, not face value) of the 5-year zero-coupon bond (maturing `Nov 2025`.)
* Short `$100` million (market value, not face value) of the 1-year zero-coupon bond (maturing `Nov 2021`.)
* Assume there is a `2%` haircut on each side of the trade, so it requires `$4` million of investor capital to initiate it.

1. Calculate the total profit and loss year-by-year.
1. Calculate the total return (`Nov 2025`) on the initial \\$4 million of investor capital.

#### Short position
* Each year you will roll over the short position to maintain a short `$100` million (market value) in the 1-year bond.
* This will require injecting more cash into the trade, as the expiring short will require more than `$100` million to close out. 
* In `Nov 2024`, no need to open a new short position, as your long position will (at that point) be a one-year bond.

#### Alternatives
The scheme above is for simplicity. You could try more interesting ways of setting the short position...
* Open a new short position sized to whatever is needed to cover the expiring short position.
* Set the short positions to duration-hedge the long position.

In [3]:
NOTIONAL = 100e6 # starting with 100 million long and short
HAIRCUT = 0.02
EQUITY0 = 2 * HAIRCUT * NOTIONAL  # should be $4mm

dates = list(nov.index)

# Face of the long 5-year zero bought in Nov 2020
face_long = NOTIONAL / nov.loc[dates[0], 5]

rows = []
equity = EQUITY0

for i in range(5):  # 2020->2021, ..., 2024->2025
    t0, t1 = dates[i], dates[i+1]
    rem0 = 5 - i  # remaining maturity at t0 (5,4,3,2,1)
    rem1 = rem0 - 1

    # Long: mark-to-market over the year (5y->4y, ..., 1y->maturity)
    mv_long_0 = face_long * nov.loc[t0, rem0]
    mv_long_1 = face_long * (nov.loc[t1, rem1] if rem1 > 0 else 1.0)
    pnl_long = mv_long_1 - mv_long_0

    # Short: roll the 1-year short for years starting 2020..2023 only (we stop after Nov 2024)
    if i <= 3:
        price_1y = nov.loc[t0, 1]
        face_short = NOTIONAL / price_1y
        # P&L of the expiring short over the year: (-face at maturity) - (-100mm initial MV)
        pnl_short = NOTIONAL - face_short
    else:
        pnl_short = 0.0

    pnl_total = pnl_long + pnl_short
    equity += pnl_total

    rows.append({
        "start": t0.date(),
        "end": t1.date(),
        "long_pnl_$": pnl_long,
        "short_pnl_$": pnl_short,
        "total_pnl_$": pnl_total,
        "equity_if_no_new_capital_$": equity
    })

pnl_11 = pd.DataFrame(rows)
display(pnl_11)

final_equity = pnl_11.iloc[-1]["equity_if_no_new_capital_$"]
total_return_on_4mm = final_equity / EQUITY0 - 1
print(f"Final equity (no new capital added): ${final_equity:,.0f}")
print(f"Total return on initial $4mm: {total_return_on_4mm:.2%}")


,start,end,long_pnl_$,short_pnl_$,total_pnl_$,equity_if_no_new_capital_$
0,2020-11-30,2021-11-30,-2.362973e+06,-1.162275e+05,-2.479200e+06,1.520800e+06
1,2021-11-30,2022-11-30,-7.204655e+06,-2.483473e+05,-7.453003e+06,-5.932203e+06
2,2022-11-30,2023-11-30,2.480175e+06,-4.780616e+06,-2.300442e+06,-8.232644e+06
3,2023-11-30,2024-11-29,4.812566e+06,-5.143642e+06,-3.310760e+05,-8.563720e+06
4,2024-11-29,2025-11-28,4.158168e+06,0.000000e+00,4.158168e+06,-4.405552e+06


Final equity (no new capital added): $-4,405,552
Total return on initial $4mm: -210.14%


### 1.1  What happened in this carry trade?

The idea here is to earn the difference between long-term and short-term interest rates as long as bond prices don’t move much.

The large loss we see comes from what actually happened to interest rates between 2021 and 2022. Rates rose sharply across the entire yield curve. When rates rise, bond prices fall — and **long-term bonds fall much more than short-term bonds** because they are more sensitive to interest rates.

That means:
- The **5-year bond we owned lost a lot of value**, and
- The **1-year bond we shorted did not protect us much**, even though its funding cost increased.

Because this trade was very **highly leveraged** (only $4 million supporting $200 million of positions), these price moves were large enough to wipe out the initial capital and more.

So the big loss is not a mistake — it reflects the fact that carry trades make money only when rates are stable, and they suffer large losses when rates jump, as they did in 2021–2022.

Also, the loss probably wouldn't be as large because we would have forcibly closed the position earlier.



## 1.2

How would this trade play out if the path of one-year spot rates equaled the forward rates observed in `2020`?

### Counterfactual: realized 1-year spot rates follow the Nov 2020 forward curve

Let $Z_n(0)$ be the Nov 2020 price (discount factor) of the $n$-year zero.

Then the discount factor from year $t$ to year $t+n$ implied by Nov 2020 is:

$$
Z_n(t) \equiv \frac{Z_{t+n}(0)}{Z_t(0)}.
$$

Interpretation: if the future short-rate path exactly matches the forward rates seen in 2020 (and term premia don’t change), then the entire future curve is “pre-determined” by the original discount factors.
We can therefore compute the **implied** Nov 2021, Nov 2022, ... bond prices from Nov 2020 alone.


In [6]:
# Nov 2020 discount factors (prices) Z1..Z5
Z0 = nov.loc[dates[0]]
Z = [1.0] + [float(Z0[k]) for k in [1,2,3,4,5]]  # Z[0]=1, Z[1]=Z1, ..., Z[5]=Z5

# implied price at time t for an n-year zero: Z_{t+n}(0) / Z_t(0)
implied = pd.DataFrame(index=dates, columns=[1,2,3,4,5], dtype=float)
for t in range(6):          # t=0..5
    for n in range(1, 6):   # n=1..5
        if t + n <= 5:
            implied.loc[dates[t], n] = Z[t+n] / Z[t]

display(implied)


,1,2,3,4,5
2020-11-30,0.998839,0.997047,0.994464,0.988809,0.981515
2021-11-30,0.998206,0.995620,0.989958,0.982656,NaN
2022-11-30,0.997409,0.991738,0.984422,NaN,NaN
2023-11-30,0.994314,0.986979,NaN,NaN,NaN
2024-11-29,0.992624,NaN,NaN,NaN,NaN
2025-11-28,NaN,NaN,NaN,NaN,NaN


In [7]:
# Rerunning the same strategy logic using implied prices instead of realized prices
face_long_imp = NOTIONAL / implied.loc[dates[0], 5]

rows = []
equity = EQUITY0

for i in range(5):
    t0, t1 = dates[i], dates[i+1]
    rem0 = 5 - i
    rem1 = rem0 - 1

    mv_long_0 = face_long_imp * implied.loc[t0, rem0]
    mv_long_1 = face_long_imp * (implied.loc[t1, rem1] if rem1 > 0 else 1.0)
    pnl_long = mv_long_1 - mv_long_0

    if i <= 3:
        face_short = NOTIONAL / implied.loc[t0, 1]
        pnl_short = NOTIONAL - face_short
    else:
        pnl_short = 0.0

    pnl_total = pnl_long + pnl_short
    equity += pnl_total

    rows.append({
        "start": t0.date(),
        "end": t1.date(),
        "long_pnl_$": pnl_long,
        "short_pnl_$": pnl_short,
        "total_pnl_$": pnl_total,
        "equity_$": equity
    })

pnl_12 = pd.DataFrame(rows)
display(pnl_12)

print(f"Final equity under forward-rate path: ${equity:,.0f}")
print(f"Total return on initial $4mm: {equity/EQUITY0 - 1:.2%}")


,start,end,long_pnl_$,short_pnl_$,total_pnl_$,equity_$
0,2020-11-30,2021-11-30,116227.507512,-116227.507512,0.000000,4.000000e+06
1,2021-11-30,2022-11-30,179949.834163,-179740.925765,208.908398,4.000209e+06
2,2022-11-30,2023-11-30,260517.654880,-259748.339154,769.315726,4.000978e+06
3,2023-11-30,2024-11-29,575070.015938,-571886.353224,3183.662714,4.004162e+06
4,2024-11-29,2025-11-28,751516.145652,0.000000,751516.145652,4.755678e+06


Final equity under forward-rate path: $4,755,678
Total return on initial $4mm: 18.89%


### 1.2  What happened under the forward-rate path?

In this experiment, we force the one-year interest rate each year to equal the forward rates implied by the yield curve in Nov 2020. This removes interest-rate surprises: the yield curve evolves exactly as the market expected in 2020.

Under this path, the carry trade makes money. The 5-year bond gradually rolls down the yield curve into shorter-maturity bonds with lower yields, so its price rises each year. At the same time, the 1-year funding rate remains lower than the yield locked in on the long bond.

As a result, the trade earns the carry embedded in the 2020 yield curve, producing a positive final return of about **19% on the initial $4 million**.

This shows that the carry trade would have worked if rates had followed the path implied by the forward curve. The large loss in 1.1 came not from the carry itself, but from unexpected rate hikes that caused long-term bond prices to fall.


## 1.3

Given Fact 3 of the *dynamic* (conditional) tests of the Expectations Hypothesis (EH), do you expect that as of `Nov 2025` the long-short trade above looks more or less favorable for `Nov 2025-2030` than it did for `Nov 2020-2025`?

In [9]:
prices = pd.read_excel("famabliss_strips_2025-11-28.xlsx", sheet_name="prices")
prices["date"] = pd.to_datetime(prices["date"])
prices = prices.set_index("date")

# Extract Nov 2020 and Nov 2025
p20 = prices.loc[pd.Timestamp("2020-11-30")]
p25 = prices.loc[pd.Timestamp("2025-11-28")]

# ---- Forward spread formula for the 5y vs 1y trade ----
# f^(4→5) = ln(P4) - ln(P5)
# y^(1)   = - ln(P1)
# forward spread = f^(4→5) - y^(1)

def forward_spread(row):
    P1, P4, P5 = row[1], row[4], row[5]
    f45 = np.log(P4) - np.log(P5)
    y1  = -np.log(P1)
    return f45 - y1
p25 = prices.loc[pd.Timestamp("2025-11-28")]

fs_2025 = forward_spread(p25)

print("Forward spread Nov 2020:", fs_2020)
print("Forward spread Nov 2025:", fs_2025)



Forward spread Nov 2020: 0.006241984874082134
Forward spread Nov 2025: 0.0020768464628717967


In this answer we are giving the ex-ante comparison rather than the realized path of the actual trade in 2020.

Under the dynamic Expectations Hypothesis, the attractiveness of a bond carry trade is measured by the **forward spread**

$$
\tilde f_t^{(4\to5)} = f_t^{(4\to5)} - y_t^{(1)},
$$

which predicts **excess returns on long-maturity bonds**.

From the Fama-Bliss zero-coupon prices:

- **Nov-2020:**
$$
\tilde f^{(4\to5)}_{2020} = 0.00624
$$

- **Nov-2025:**
$$
\tilde f^{(4\to5)}_{2025} = 0.00208
$$

The forward spread is substantially smaller in 2025 than in 2020. This means the **term premium embedded in the yield curve is much lower in Nov-2025**, so investors are being paid less to hold long-maturity bonds relative to rolling 1-year bonds.

**Conclusion:**
The carry trade is **less favorable in Nov-2025 than in Nov-2020**.